In [1]:
# use pandas to load data and visualize
# use longformer tokenizer (efficient at tokenizing large texts)
# Set Up the Model for Fine-Tuning (transition to Colab here?)
# Define Training Arguments
from google.colab import drive
drive.mount('/content/drive')
#!pip install -r '/content/drive/My Drive/CS4765 Datasets/requirements.txt'


Mounted at /content/drive


In [2]:
!pip install datasets transformers torch # was having a lot of trouble making the requirements.txt file work so I am manually adding them here as a last resort

#!pip install -r '/content/drive/My Drive/CS4765 Datasets/requirements.txt'

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [4]:
# Visualizing the data with pandasimport pandas as pd
import pandas as pd
import json

data = pd.read_json('/content/drive/My Drive/CS4765 Datasets/subtaskA_train_monolingual.jsonl', lines=True)
print(data.head())

KeyboardInterrupt: 

In [26]:
# # tokenizer function
# from transformers import LongformerTokenizer


# tokenizer = LongformerTokenizer.from_pretrained("allenai/longformer-base-4096")
# # #print(tokenizer("Hello world")["input_ids"])
# # print(tokenizer('./datasets/subtaskA_dev_monlingual.jsonl')["input_ids"])
# def tokenize_function(examples):
#     return tokenizer(examples['text'], padding='max_length', truncation=True, max_length=4096)

In [27]:
# from datasets import Dataset

# dataset = Dataset.from_pandas(data)
# tokenized_dataset = dataset.map(tokenize_function, batched=True)
# print(tokenized_dataset)

Map:   0%|          | 0/119757 [00:00<?, ? examples/s]

Dataset({
    features: ['text', 'label', 'model', 'source', 'id', 'input_ids', 'attention_mask'],
    num_rows: 119757
})


In [8]:
# Testing loading dataset
# Commented out, it passed
# from datasets import load_dataset


# raw_datasets = load_dataset('json',data_files='/content/drive/My Drive/CS4765 Datasets/subtaskA_train_monolingual.jsonl')
# rawvalidation_dataset = load_dataset('json',data_files='/content/drive/My Drive/CS4765 Datasets/validation.jsonl')

# print(raw_datasets['train'][:5])
# print(rawvalidation_dataset['train'][:5])
tokenized_subset = raw_datasets["train"].select(range(5)).map(tokenize_function, batched=True)
print(tokenized_subset)

Map:   0%|          | 0/5 [00:00<?, ? examples/s]

Dataset({
    features: ['text', 'label', 'model', 'source', 'id', 'input_ids', 'attention_mask'],
    num_rows: 5
})


In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset
import os

os.environ["WANDB_MODE"] = "disabled" # disable wandb mode because it keeps asking for an API key

# Load raw dataset
raw_datasets = load_dataset('json',data_files='/content/drive/My Drive/CS4765 Datasets/subtaskA_train_monolingual.jsonl')  # For training
# For validation while training, we took about 20% of the train data and put it into a seperate file
rawvalidation_dataset = load_dataset('json',data_files='/content/drive/My Drive/CS4765 Datasets/validation.jsonl')

# Load Longformer tokenizer and model
model_name = "allenai/longformer-base-4096"
tokenizer = AutoTokenizer.from_pretrained(model_name, model_max_length=4096)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

# Tokenize the dataset using Longformer tokenizer
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_dataset_train = raw_datasets["train"].map(tokenize_function, batched=True)
tokenized_validation = rawvalidation_dataset["train"].map(tokenize_function, batched=True)

# Define training arguments with defaulrt values
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    save_strategy="epoch",
    save_steps=500,  # Save every 500 in case we get booted out for some reason I read we can use `trainer.train(resume_from_checkpoint=True)`
    learning_rate=2e-5,
    per_device_train_batch_size=4,  # Lower batch size for Longformer
    num_train_epochs=3,
    weight_decay=0.01,
    save_total_limit=1,
    logging_dir="./logs",
    load_best_model_at_end=True,
    log_level="debug",
)

# Create Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset_train,
    eval_dataset=tokenized_validation,
)

# Train the model
trainer.train() # resume_from_checkpoint=True, we can add this in here in the case of failure

# Save it so we don't need to retrain
print("Saving model...")
model.save_pretrained("./longformer_model")
print("Model saved.")

print("Saving tokenizer...")
tokenizer.save_pretrained("./longformer_model")
print("Tokenizer saved.")



Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/694 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

...

pytorch_model.bin:   0%|          | 0.00/597M [00:00<?, ?B/s]

Some weights of LongformerForSequenceClassification were not initialized from the model checkpoint at allenai/longformer-base-4096 and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/102525 [00:00<?, ? examples/s]

Map:   0%|          | 0/17232 [00:00<?, ? examples/s]

Currently training with a batch size of: 4
The following columns in the training set don't have a corresponding argument in `LongformerForSequenceClassification.forward` and have been ignored: id, text, source, model. If id, text, source, model are not expected by `LongformerForSequenceClassification.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 102,525
  Num Epochs = 3
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 76,896
  Number of trainable parameters = 148,660,994
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
Initializing global attention on CLS

In [3]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset
import os
import torch
torch.cuda.get_device_name(0) ## make sure we are using the mega gpu


os.environ["WANDB_MODE"] = "disabled"  # Disable wandb mode

# Load raw dataset
raw_datasets = load_dataset('json', data_files='/content/drive/My Drive/CS4765 Datasets/subtaskA_train_monolingual.jsonl')
rawvalidation_dataset = load_dataset('json', data_files='/content/drive/My Drive/CS4765 Datasets/validation.jsonl')

# Load DistilBERT tokenizer and model
model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

# Tokenize the dataset using DistilBERT tokenizer
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_dataset_train = raw_datasets["train"].map(tokenize_function, batched=True)
tokenized_validation = rawvalidation_dataset["train"].map(tokenize_function, batched=True)

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    save_total_limit=1,
    logging_dir="./logs",
    load_best_model_at_end=True,
    log_level="debug",
)

# Create Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset_train,
    eval_dataset=tokenized_validation,
)

# Train the model
try:
    trainer.train()
except Exception as e:
    print(f"Training error: {e}")

# Save it so we don't need to retrain
print("Saving model...")
model.save_pretrained("./distilbert_model")
print("Model saved.")

print("Saving tokenizer...")
tokenizer.save_pretrained("./distilbert_model")
print("Tokenizer saved.")


Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

...

Epoch,Training Loss,Validation Loss
1,0.060700,3.370243
2,0.027500,4.799409
3,0.007600,5.879086


The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text, model, source, id. If text, model, source, id are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.

***** Running Evaluation *****
  Num examples = 17232
  Batch size = 8
Saving model checkpoint to ./results/checkpoint-6408
Configuration saved in ./results/checkpoint-6408/config.json
Model weights saved in ./results/checkpoint-6408/model.safetensors
The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text, model, source, id. If text, model, source, id are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.

***** Running Evaluation *****
  Num examples = 17232
  Batch size = 8
Saving model checkpoint to ./results/checkpoint-12816
Confi

Saving model...


Model weights saved in ./distilbert_model/model.safetensors
tokenizer config file saved in ./distilbert_model/tokenizer_config.json
Special tokens file saved in ./distilbert_model/special_tokens_map.json


Model saved.
Saving tokenizer...
Tokenizer saved.


In [1]:
# If we wnat to reuse the model we trained we can use this code now
# from transformers import AutoTokenizer, AutoModelForSequenceClassification

# # Load the saved model and tokenizer
# model = AutoModelForSequenceClassification.from_pretrained("./longformer_model")
# tokenizer = AutoTokenizer.from_pretrained("./longformer_model")

# Example
# text = "Your input text here"
# inputs = tokenizer(text, return_tensors="pt", truncation=True, padding="max_length", max_length=4096)
# outputs = model(**inputs)
# predictions = outputs.logits.argmax(dim=-1)
# print(f"Prediction: {predictions.item()}")

from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

# Load the saved model and tokenizer
model = AutoModelForSequenceClassification.from_pretrained("./distilbert_model")
tokenizer = AutoTokenizer.from_pretrained("./distilbert_model")


# Load a test dataset
raw_test_dataset = load_dataset('json', data_files='/content/drive/MyDrive/CS4765 Datasets/subtaskA_dev_monolingual.jsonl')

# Tokenize the test dataset
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_test = raw_test_dataset["train"].map(tokenize_function, batched=True)
# Evaluate on the test dataset
results = trainer.evaluate(eval_dataset=tokenized_test)
print("Evaluation Results:", results)



FileNotFoundError: [Errno 2] No such file or directory: './longformer_model'

In [6]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

# Load the saved model and tokenizer
model = AutoModelForSequenceClassification.from_pretrained("./distilbert_model")
tokenizer = AutoTokenizer.from_pretrained("./distilbert_model")

# Example input
example_text = "Artificial intelligence is revolutionizing many industries by enabling automation and deeper insights through data analysis."

# Tokenize the input
inputs = tokenizer(example_text, return_tensors="pt", padding="max_length", truncation=True)

# Move tensors to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
inputs = {key: value.to(device) for key, value in inputs.items()}

# Perform prediction
outputs = model(**inputs)
logits = outputs.logits
predicted_class = torch.argmax(logits, dim=1).item()

print(f"Predicted Class: {predicted_class}")


loading configuration file ./distilbert_model/config.json
Model config DistilBertConfig {
  "_name_or_path": "./distilbert_model",
  "activation": "gelu",
  "architectures": [
    "DistilBertForSequenceClassification"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "problem_type": "single_label_classification",
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "torch_dtype": "float32",
  "transformers_version": "4.46.2",
  "vocab_size": 30522
}

loading weights file ./distilbert_model/model.safetensors
All model checkpoint weights were used when initializing DistilBertForSequenceClassification.

All the weights of DistilBertForSequenceClassification were initialized from the model checkpoint at ./distilbert_model.
If your task is similar to 

Predicted Class: 1


In [1]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# Load the trained model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("./longformer_model")
model = AutoModelForSequenceClassification.from_pretrained("./longformer_model")

# Test on a sample text
sample_text = "This is an example sentence to classify."
inputs = tokenizer(sample_text, return_tensors="pt", padding="max_length", truncation=True, max_length=4096)
outputs = model(**inputs)
predictions = outputs.logits.argmax(dim=-1)
print(f"Predicted label: {predictions.item()}")


OSError: Incorrect path_or_model_id: './longformer_model'. Please provide either the path to a local folder or the repo_id of a model on the Hub.